
<img src="img/logo_wiwi.png" width="25%" align="left">

<img src="img/decision_analytics_logo.png" width="17%" align="right">



<br><br><br><br><br><br><br><br>



# Algorithmen und Datenstrukturen(A+D)-Projekt 

**Sommersemester 2022**


# 7. Experimente

<br>

<br>
<br>

**J-Prof. Dr. Michael Römer, Till Porrmann, Jakob Schulte, Henning Witteborg**

Juniorprofessur für Decision Analytics  | Universität Bielefeld

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numba import njit

## Was machen wir heute?

**Wir betrachten ein paar hilfreiche Techniken für das Durchführen und Auswerten von Experimenten mit Algorithmen**

- Aufrufen mehrerer Instanzen und Algorithmen
- Datenstrukturen für Ergebnisse
- Aufbereiten der Ergebnisse mit data frames
- Speichern von Ergebnissen in Dateien
- Auslesen von Ergebnissen aus Dateien








# Rückblick: Algorithmen für das TSP

## Das TSP als Beispiel

Für das TSP haben wir 
- für viele Instanzen optimale Zielfunktionswerte: http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html
- das Python-Paket `python-tsp`, mit dem wir unsere Algorithmen vergleichen können (https://github.com/fillipe-gsm/python-tsp)
- einige Algorithmen implementiert
  - Nearest Neighbor
  - Multistart Nearest Neighbor
  - Rollout mit Nearest Neighbor
  - Simplified Rollout
  - Multistep-Lookahead mit (Simplified) Rollout

**Wie können wir**
- sinnvoll mit mehreren (z.B. > 10) Instanzen experimentieren?
- die Ergebnisse in Dateien speichern und aufbereiten?

In [2]:
from python_tsp.distances import tsplib_distance_matrix

#tsplib_file = "./../problems/tsp/instances/gr48.tsp" # optimale Lösung 5046 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
#tsplib_file = "./../problems/tsp/instances/brazil58.tsp" # optimale Lösung 25395 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
tsplib_file = "./../problems/tsp/instances/berlin52.tsp" # optimale Lösung 7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)

distance_matrix = tsplib_distance_matrix(tsplib_file)

In [3]:
from python_tsp.heuristics import solve_tsp_simulated_annealing

permutation, distance_simulated_annealing = solve_tsp_simulated_annealing(distance_matrix)
distance_simulated_annealing

7542

## Hilfsfunktion: `select_nearest_neighbor`

- wir ändern / vereinfachen unsere bisherige Implementierung der Hilfsfunktion
- wir brauchen eigentlich neben der Distanzmatrix nur die Permutationsliste
  - das letzte Element ist der Knoten, von dem aus gesucht wird
- wir nutzen wieder `njit` von numba (könnte man auch weglassen, wäre dann aber langsamer)

In [4]:
@njit
def select_nearest_neighbor(permutation, distance_matrix ):
    
    # node ist der letzte Knoten der Permutation
    # wir könnten auch permutation[-1] schreiben, aber das funktioniert mit Numba nicht
    node = permutation[len(permutation)-1]
    
    smallest_distance = 9999999999 ## grosser Wert
    nearest_neighbor = 0
    
    #Anzahl an Knoten = Dimension der Distanzmatrix
    for neighbor in range(len(distance_matrix)):
        
        if neighbor in permutation:
            continue
            
        if distance_matrix[node][neighbor] < smallest_distance:
            nearest_neighbor = neighbor
            smallest_distance = distance_matrix[node][neighbor]            
       
    return nearest_neighbor, smallest_distance 

## Implementierung des Nearest-Neighbor-Algorithmus
- wir nutzen nun die neue Funktion in der Implementierung des Nearest Neighbor-Algorithmus
- auch hier haben wir die Implementierung etwas vereinfacht:
  - wir übergeben nur eine Permutation und die Distanzmatrix

- beim alleinigen Aufruf des Algorithmus übergeben wir eine Liste mit einem Startknoten
- bei Verwendung im Rollout-Algorithmus übergeben wir die bisherige Permutation


In [5]:
@njit
def tsp_nearest_neighbor(permutation, distance_matrix ):
    
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(permutation) < len(distance_matrix):
        
        node, distance = select_nearest_neighbor(permutation, distance_matrix )
        
        permutation.append(node)
        total_distance += distance
        
    total_distance += distance_matrix[permutation[len(permutation)-1], permutation[0]]
    return permutation, total_distance


..probieren wir es aus:

In [6]:
permutation, distance_nearest_neighbor = tsp_nearest_neighbor([0], distance_matrix)
print(distance_nearest_neighbor)


C:\Users\Michael\AppData\Local\Temp\ipykernel_21072\4184130370.py:9: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'lst' of function 'in_seq.<locals>.seq_contains_impl'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\miniconda3\envs\audproj\lib\site-packages\numba\cpython\listobj.py", line 662:
def in_seq(context, builder, sig, args):
    def seq_contains_impl(lst, value):
    ^

  node, distance = select_nearest_neighbor(permutation, distance_matrix )
C:\Users\Michael\AppData\Local\Temp\ipykernel_21072\4184130370.py:9: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'select_nearest_neighbor'.

For more information visit https://numba.pydata.org/numba-doc/latest/r

8980


## Eine Routine zum Evaluieren einer Lösung

- immer wenn man nicht-triviale Algorithmen für Optimierungsprobleme entwickelt, sollte man einen "Solution-Checker" schreiben / nutzen

**Im Fall des TSP gilt es:**
- zu prüfen, ob
  - die Lösung die Richtige Anzahl an Knoten enthält
  - dass es sich bei der Lösung tatsächlich um eine Permutation der Indizes handelt (kein Index kommt zweimal vor)
- die Distanz der Tour zu berechnen

In [7]:
def evaluate_tsp_solution(distance_matrix, permutation):
    n = len(distance_matrix)
    if len(permutation) != n:
        print ("Wrong number of nodes")
        return -1
     # Menge der Lösungsindizes muss = der Menge der Indizes von 0 bis n-1 sein
    if set(permutation) != set(range(n)):
        print ("Not a proper permutation!")
        return  -1
    
    total_distance = 0
    for i in range(n):
        if i < n-1:
            total_distance += distance_matrix[permutation[i],permutation[i+1]]
        else:
            total_distance += distance_matrix[permutation[i],permutation[0]]   
            
    return total_distance    

## Erweiterung: Evaluation zum Überprüfen einer gegebenen Distanz

- oftmals ist es nützlich, in einer Funktion direkt die vom Algorithmus berechnete Distanz zu prüfen
- folgende Funktion macht eine entsprechende Ergebnisausgabe:


In [8]:
def print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance):
    
    eval_distance = evaluate_tsp_solution(distance_matrix, permutation)
    
    if distance == eval_distance:
        print ("Solution feasible, distance is: ", distance)
    elif eval_distance < 0:
        print("Solution infeasible")
    else: 
        print("Solution feasible, wrong distance: ", distance, " evaluation gave ", eval_distance)

In [9]:
permutation_nearest_neighbor, distance_nearest_neighbor = tsp_nearest_neighbor([0], distance_matrix)
print_obj_and_eval_tsp_solution(distance_matrix, permutation_nearest_neighbor, distance_nearest_neighbor )

Solution feasible, distance is:  8980


## Multistart-Nearest Neighbor

- wir hatten gesehen, dass es sich lohnen kann, an allen möglichen Knoten zu starten:

In [10]:
def tsp_multistart_nearest_neighbor(distance_matrix):
    best_permutation = []
    best_distance = 999999
    
    for i in range(len(distance_matrix)):
        permutation, distance = tsp_nearest_neighbor([i], distance_matrix)
        
        if distance < best_distance:
            best_permutation = permutation
            best_distance = distance
    
    return best_permutation, best_distance
    

In [11]:
permutation_multistart_nearest_neighbor, distance_multistart_nearest_neighbor = tsp_multistart_nearest_neighbor( distance_matrix)
print_obj_and_eval_tsp_solution(distance_matrix, permutation_multistart_nearest_neighbor, distance_multistart_nearest_neighbor)

Solution feasible, distance is:  8181


## Nearest Neighbor vs Rollout
- beim Nearest Neighbor wählen wir als nächsten Knoten den mit der kürzesten Distanz zum letzten Knoten in der partiellen Tour:

<img src="./img/single_trajectory.png" width="50%">

- beim Rollout wählen wir den mit dem kleinsten Q-Faktor, bestehend aus der Summe von
  - Distanz zum letzen Knoten
  - Länge der Rest-Tour vom letzten Knoten aus

<img src="./img/rollout_general.png" width="50%">

## Rollout


In [12]:
@njit
def select_using_rollout_nn(permutation, distance_matrix):
    
    node = permutation[len(permutation)-1] 
    best_q_value = 1000000
    best_node = node        
        
    for next_node in range(len(distance_matrix)):
        if next_node in permutation: 
            continue            
        # _, heißt, dass wir den ersten Rückgabewert ignorieren
        
        # Berechne die NN-Länge der Rest-Tour von next_node aus
        _, nn_value = tsp_nearest_neighbor(permutation + [next_node], distance_matrix)
        
        q_value = distance_matrix[node,next_node] + nn_value

        if q_value < best_q_value:
            best_node = next_node
            best_q_value = q_value

    
   
    return best_node, distance_matrix[node,best_node]


## Die Hauptfunktion:

In [13]:
@njit
def tsp_rollout_nn(permutation, distance_matrix):
        
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(permutation) < len(distance_matrix):    
        
        next_node, distance = select_using_rollout_nn(permutation, distance_matrix)
        permutation.append(next_node)
        total_distance += distance     
        
    total_distance += distance_matrix[permutation[len(permutation)-1],permutation[0]]
    return permutation, total_distance

..probieren wir es wieder aus:

In [14]:
permutation_rollout_nn, distance_rollout_nn = tsp_rollout_nn([0], distance_matrix)
print_obj_and_eval_tsp_solution(distance_matrix, permutation_rollout_nn, distance_rollout_nn)



C:\Users\Michael\AppData\Local\Temp\ipykernel_21072\1017998703.py:9: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'select_using_rollout_nn'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\AppData\Local\Temp\ipykernel_21072\3105444862.py", line 1:
<source missing, REPL/exec in use?>

  next_node, distance = select_using_rollout_nn(permutation, distance_matrix)
C:\Users\Michael\miniconda3\envs\audproj\lib\site-packages\numba\core\ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'tsp_rollout_nn'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-li

Solution feasible, distance is:  8042


## Übung: Multistart Rollout


><div class="alert alert-block alert-info">
<b>Implementieren Sie eine Multistart-Rollout-Funktion! </b></div>  

In [15]:
def tsp_multistart_rollout_nn(distance_matrix):
    best_permutation = []
    best_distance = 999999
    
    for i in range(len(distance_matrix)):
        permutation, distance = tsp_rollout_nn([i], distance_matrix)
        
        if distance < best_distance:
            best_permutation = permutation
            best_distance = distance
    
    return best_permutation, best_distance

In [16]:
permutation_multistart_rollout_nn, distance_multistart_rollout_nn = tsp_multistart_rollout_nn( distance_matrix)
print_obj_and_eval_tsp_solution(distance_matrix, permutation_multistart_rollout_nn, distance_multistart_rollout_nn)



Solution feasible, distance is:  7819


## Simplified Rollout

- wir wollen nun auch die Idee des Simplified Rollout übertragen in die neue Form der Implementierung

><div class="alert alert-block alert-info">
<b>Wo müssen die größeren Änderungen durchgeführt werden - in der `select`-Hilfsfunktion oder in der Hauptfunktion?</b></div>  

## Simplified Rollout: Die Select-Funktion

In [17]:
@njit
def select_using_simplified_rollout_nn(permutation, distance_matrix,  max_number_of_neighbors_rollout):
    
    node = permutation[len(permutation)-1] 
                                    
    best_q_value = 1000000
    best_node = node
   
    sorted_neighbors = np.argsort(distance_matrix[node])
    
    number_of_neighbors_rollout = 0
    for next_node in sorted_neighbors:
        if next_node in permutation: 
            continue            
       
        
        number_of_neighbors_rollout += 1
        # zähler wenn 
        if number_of_neighbors_rollout > max_number_of_neighbors_rollout:
            break     
                                    
                
        _, nn_value = tsp_nearest_neighbor(permutation + [next_node], distance_matrix)
        
        q_value = distance_matrix[node,next_node] + nn_value

        if q_value < best_q_value:
            best_node = next_node
            best_q_value = q_value

    return best_node, distance_matrix[node,best_node]



## Simplified Rollout: Die Hauptfunktion

In [18]:
@njit
def tsp_simplified_rollout_nn(permutation, distance_matrix, max_number_of_neighbors_rollout):
        
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(permutation) < len(distance_matrix):    
        
        next_node, distance = select_using_simplified_rollout_nn(permutation, distance_matrix, max_number_of_neighbors_rollout)
        permutation.append(next_node)
        total_distance += distance     
        
    total_distance += distance_matrix[permutation[len(permutation)-1],permutation[0]]
    return permutation, total_distance

...probieren wir es aus:

In [19]:
max_number_of_neighbors_rollout = 10
permutation_simplified_rollout_nn, distance_simplified_rollout_nn = tsp_simplified_rollout_nn([0], distance_matrix, max_number_of_neighbors_rollout)
print_obj_and_eval_tsp_solution(distance_matrix, permutation_simplified_rollout_nn, distance_simplified_rollout_nn)


C:\Users\Michael\AppData\Local\Temp\ipykernel_21072\3887431228.py:9: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'select_using_simplified_rollout_nn'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\AppData\Local\Temp\ipykernel_21072\739218588.py", line 1:
<source missing, REPL/exec in use?>

  next_node, distance = select_using_simplified_rollout_nn(permutation, distance_matrix, max_number_of_neighbors_rollout)
C:\Users\Michael\miniconda3\envs\audproj\lib\site-packages\numba\core\ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'tsp_simplified_rollout_nn'.

For more information visit https://numba.pydata.org/numba-doc/l

Solution feasible, distance is:  8122


## Multi-Step-Lookahead

**Kernidee:**
- wir parametrisieren die Anzahl der Lookahead-Schritte
- und rufen den lookahead-Algorithmus rekursiv mit jeweils um 1 reduzierter Anzahl an Lookahead-Schritten auf!

- hier direkt die Simplified-Variante

In [20]:
@njit
def select_using_simplified_multi_step_lookahead(permutation, distance_matrix, max_number_of_neighbors, number_of_lookahead_steps):
    node = permutation[-1] 
    best_q_value = 1000000
    best_node = node
    
    number_of_lookahead_steps = min(number_of_lookahead_steps, len(distance_matrix) - len(permutation))
    
    sorted_neighbors = np.argsort(distance_matrix[node])
    
    number_of_neighbors_rollout = 0
    
    for next_node in sorted_neighbors:
        if next_node in permutation: 
            continue            
       
        
        number_of_neighbors_rollout += 1
        # zähler wenn 
        if number_of_neighbors_rollout > max_number_of_neighbors:
            break                     
            

        if number_of_lookahead_steps > 1:
            _, value = tsp_rollout_nn_simplified_multi_step_lookahead(permutation + [next_node],
                                                                      distance_matrix, 
                                                                      max_number_of_neighbors,
                                                                      number_of_lookahead_steps - 1)
        else:
            _, value = tsp_rollout_nn(permutation + [next_node], distance_matrix)

        q_value = distance_matrix[node,next_node] + value

        if q_value < best_q_value:
            best_node = next_node
            best_q_value = q_value
    
   
    return best_node, distance_matrix[node,best_node]

    

In [21]:
@njit
def tsp_rollout_nn_simplified_multi_step_lookahead(permutation, distance_matrix, max_number_of_neighbors, number_of_lookahead_steps):
    

    total_distance = 0
    
    node = permutation[len(permutation)-1]    

    while len(permutation) < len(distance_matrix):    
        
        node, distance = select_using_simplified_multi_step_lookahead(permutation,
                                                                      distance_matrix,
                                                                      max_number_of_neighbors,
                                                                      number_of_lookahead_steps)
        permutation.append(node)
        total_distance += distance     
     
    
    total_distance += distance_matrix[permutation[len(permutation)-1],permutation[0]]
    return permutation, total_distance

In [22]:
max_number_of_neighbors = 3
number_of_lookahead_steps = 2
permutation_simplified_multi_step_lookahead, distance_simplified_multi_step_lookahead = tsp_rollout_nn_simplified_multi_step_lookahead([2], 
                                                                       distance_matrix,
                                                                       max_number_of_neighbors,
                                                                       number_of_lookahead_steps)
print_obj_and_eval_tsp_solution(distance_matrix, permutation_simplified_multi_step_lookahead, distance_simplified_multi_step_lookahead)

C:\Users\Michael\AppData\Local\Temp\ipykernel_21072\4193355005.py:11: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'select_using_simplified_multi_step_lookahead'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\AppData\Local\Temp\ipykernel_21072\749170160.py", line 1:
<source missing, REPL/exec in use?>

  node, distance = select_using_simplified_multi_step_lookahead(permutation,
C:\Users\Michael\miniconda3\envs\audproj\lib\site-packages\numba\core\ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'tsp_rollout_nn_simplified_multi_step_lookahead'.

For more information visit https://numba.pydata.org/numba-doc/latest/referen

Solution feasible, distance is:  7918


# Experimente und deren Auswertung

## Herausforderungen für die Auswertung

**Wir wollen:**
- nicht nur eine, sondern mehrere Instanzen testen
  - ohne jedes Mal jede Zelle des Notebooks laufen zu lassen
- die Ergebnisse in Dateien speichern
  - Läufe können länger dauern und an verschiedenen Tagen passieren
  - die Auswertung kann später gemacht wercen
- damit umgehen, dass nicht nur verschiedene Algorithmen, sondern auch Varianten und paramtrisierte Algorithmen gibt
  - z.B. Anzahl Steps beim Multistep Lookahead, Anzahl Nachbarn beim Simplified Rollout
- sinnvolle Tabellen erstellen
  - Ergebnisse und Laufzeiten
  - Umgang mit unterschiedlichen Instanzgrößen


## Mehrere Algorithmen laufen lassen

- wir schreiben eine Funktion, die 
  - für eine Instanz mehrere Algorithmen aufruft
  - und die Ergebnisse ausgibt
  

In [23]:
from timeit import default_timer as timer

def run_algorithms(instance_name):
    
    tsplib_file = "./../problems/tsp/instances/" + instance_name + ".tsp" # optimale Lösung 7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
    distance_matrix = tsplib_distance_matrix(tsplib_file)
    print ("Instanz", instance_name)
    print ("Nodes", len(distance_matrix))
    
    print("Nearest Neighbor:")
    starttime = timer()    
    permutation_nn, distance_nn = tsp_nearest_neighbor([0], distance_matrix)
    print_obj_and_eval_tsp_solution(distance_matrix, permutation_nn, distance_nn )
    print(f"Time: {timer()- starttime:0.3f}")
    
    print("Multistart Nearest Neighbor:")
    starttime = timer()    
    permutation_multistart_rollout_nn, distance_multistart_rollout_nn = tsp_multistart_rollout_nn( distance_matrix)
    print_obj_and_eval_tsp_solution(distance_matrix, permutation_multistart_rollout_nn, distance_multistart_rollout_nn)
    print(f"Time: {timer()- starttime:0.3f}")
    
    
    

In [24]:
run_algorithms("berlin52")

Instanz berlin52
Nodes 52
Nearest Neighbor:
Solution feasible, distance is:  8980
Time: 0.000
Multistart Nearest Neighbor:
Solution feasible, distance is:  7819
Time: 3.566


## Eine Funktion zum Experimentieren mit mehreren Instanzen
- wir möchten nicht  nur eine, sondern mehrere Instanzen testen

In [25]:
def run_algorithms_on_instances(instance_names):

    for instance_name in instance_names: 
        run_algorithms(instance_name)
        

run_algorithms_on_instances(["berlin52", "gr48"])  

Instanz berlin52
Nodes 52
Nearest Neighbor:
Solution feasible, distance is:  8980
Time: 0.000
Multistart Nearest Neighbor:
Solution feasible, distance is:  7819
Time: 3.510
Instanz gr48
Nodes 48
Nearest Neighbor:
Solution feasible, distance is:  6098
Time: 0.000
Multistart Nearest Neighbor:
Solution feasible, distance is:  5445
Time: 2.184


## Daten strukturiert erfassen

- im obigen Code wurden die Daten relativ unstrukturiert ausgegeben

**Neuer Ansatz:**
- wir sammeln die Daten für jeden Lauf eines Algorithmus in einem `dict` mit den folgenden Daten
  - Instance
  - Algorithm
  - Objective
  - Time
- fügen all diese Ergebnisse zusammen
- und bilden daraus einen `dataframe`
  
 

In [26]:
def run_algorithms(instance_name):
    
    
    results=[]
    tsplib_file = "./../problems/tsp/instances/" + instance_name +".tsp" # # optimale Lösung 7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
    distance_matrix = tsplib_distance_matrix(tsplib_file)
    
    print ("Instanz", instance_name)
    
    result_dict = {}
    result_dict["Instance"] = instance_name
    result_dict["Algorithm"] = "PyTSP"
    starttime = timer()    
    
    permutation, distance_simulated_annealing = solve_tsp_simulated_annealing(distance_matrix)
    result_dict["Objective"] = distance_simulated_annealing 
    result_dict["Time"] = timer()- starttime
    results.append(result_dict)
    
  
    result_dict = {}
    result_dict["Instance"] = instance_name
    result_dict["Algorithm"] = "NN"
    starttime = timer()    
    permutation_nn, distance_nn = tsp_nearest_neighbor([0], distance_matrix)
    result_dict["Objective"] = distance_nn
    result_dict["Time"] = timer()- starttime
    results.append(result_dict)
    
    result_dict = {}
    result_dict["Instance"] = instance_name
    result_dict["Algorithm"] = "MS NN"
    starttime = timer()    
    permutation_nn, distance_nn = tsp_multistart_nearest_neighbor( distance_matrix)
    result_dict["Objective"] = distance_nn
    result_dict["Time"] = timer()- starttime
    results.append(result_dict)
    
    result_dict = {}
    result_dict["Instance"] = instance_name
    result_dict["Algorithm"] = "MS Rollout"
    starttime = timer()    
    permutation_multistart_rollout_nn, distance_multistart_rollout_nn = tsp_multistart_rollout_nn( distance_matrix)
    result_dict["Objective"] = distance_multistart_rollout_nn 
    result_dict["Time"] = timer()- starttime
    results.append(result_dict)
    
    
    return results

In [27]:
results = run_algorithms("berlin52")    
    

Instanz berlin52


In [28]:
def run_algorithms_on_instances(instance_names):
    results = []
    for instance_name in instance_names: 
        results += run_algorithms(instance_name)
        
    return results

results = run_algorithms_on_instances(["berlin52", "gr48"])    

Instanz berlin52
Instanz gr48


## Ergebnisse als `dataframe`
- man kann eine Liste von `dict`s direkt in einen pandas dataframe überführen
- diesen kann man schön als Tabelle darstellen und auch weiter manipulieren zur Auswertung

In [31]:
pd.set_option("display.precision", 4)
df = pd.DataFrame(results)
df

,Instance,Algorithm,Objective,Time
0,berlin52,PyTSP,7753,6.4602
1,berlin52,NN,8980,0.0002
2,berlin52,MS NN,8181,0.0105
3,berlin52,MS Rollout,7819,6.8795
4,gr48,PyTSP,5285,10.2384
5,gr48,NN,6098,0.0002
6,gr48,MS NN,5840,0.0112
7,gr48,MS Rollout,5445,4.2586


## Der Gap zur optimalen Lösung

- wenn wir Instanzen mit unterschiedlich skalierten Zielfunktionswerten (Distanzen) haben
- besser ist es daher oft, die relative Performance eines Algorithmus zu einer Bezugsgröße zu betrachten, z.B.:
  - eine optimale Lösung
  - die Lösung eines 

In [32]:

instance_optimal_value = {"gr48":5046, "brazil58":25395, "berlin52":7542}

def get_opt_gap(distance, instance_name):
    return (distance - instance_optimal_value[instance_name]) / instance_optimal_value[instance_name]


><div class="alert alert-block alert-info">
<b>
Ergänzen Sie die Funktion `run_algorithms` so, dass der Gap zur optimalen Lösung als Ergebniswert ausgegeben wird!
</b></div>  

    

## Ergebnisse aufbereiten mit Pivot-Tabellen

- die Ergebnistabelle von oben ist etwas redundant 
- was, wenn wir z.B. nur eine Zeile je Instanz haben wollen und für jeden Algorithmus eine Spalte?

- derartige Umstellungen können wir mit so genannte Pivot-Tabellen machen:


In [33]:
#sicherstellen, dass Reihenfolge der Algorithmen beibehalten wird 
# hier werden die Strings "manuell" eingegeben, ggf. anpassen!
df["Algorithm"]=df["Algorithm"].astype(pd.api.types.CategoricalDtype(categories=['PyTSP','NN','MS NN', 'MS Rollout']))

df_pivot = df.pivot_table(index="Instance", columns="Algorithm", values=["Objective"])
df_pivot

Objective                       
Algorithm     PyTSP    NN MS NN MS Rollout
Instance                                  
berlin52       7753  8980  8181       7819
gr48           5285  6098  5840       5445

## Ergebnisse aufbereiten mit Pivot-Tabellen

- wir können auch Zielfunktionswert und Laufzeit in die Tabelle aufnehmen:

In [34]:
df_pivot_both = df.pivot_table(index="Instance", columns="Algorithm", values=["Objective","Time"])
df_pivot_both

Objective                            Time                           
Algorithm     PyTSP    NN MS NN MS Rollout    PyTSP      NN   MS NN MS Rollout
Instance                                                                      
berlin52       7753  8980  8181       7819   6.4602  0.0002  0.0105     6.8795
gr48           5285  6098  5840       5445  10.2384  0.0002  0.0112     4.2586

..oder die Mittelwerte der Zeilen mit anzeigen lassen

In [35]:

df_pivot = df.pivot_table(index="Instance", columns="Algorithm", values=["Objective"], margins=True)
df_pivot

Objective                                   
Algorithm     PyTSP    NN   MS NN MS Rollout       All
Instance                                              
berlin52       7753  8980  8181.0       7819  8183.250
gr48           5285  6098  5840.0       5445  5667.000
All            6519  7539  7010.5       6632  6925.125

## Ausgabe in LaTeX-Tabelle
- falls Sie LaTeX nutzen, können Sie direkt LaTeX-Code zur Darstellung der Tabelle erzeugen:

In [36]:
print(df_pivot.to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} & \multicolumn{5}{l}{Objective} \\
Algorithm &     PyTSP &    NN &   MS NN & MS Rollout &       All \\
Instance &           &       &         &            &           \\
\midrule
berlin52 &      7753 &  8980 &  8181.0 &       7819 &  8183.250 \\
gr48     &      5285 &  6098 &  5840.0 &       5445 &  5667.000 \\
All      &      6519 &  7539 &  7010.5 &       6632 &  6925.125 \\
\bottomrule
\end{tabular}



C:\Users\Michael\AppData\Local\Temp\ipykernel_21072\3636991984.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_pivot.to_latex())


## Schreibe in und Lesen aus Dateien

- oftmals können nicht alle Experimente und Auswertungen am Stück durchgeführt werden
- weiterhin möchte man häufig später weitere Analysen machen und die Auswertungen auch später noch nachvollziehen können
- mit dataframes geht das sehr leicht:

- schreiben des (ursprünglichen) data frames in eine Datei:

In [37]:
file_name = "results.txt"
df.to_csv(file_name, index=False) # csv heisst comma-separated values

- einlesen in ein Data Frame:

In [ ]:
pd.read_csv(file_name)

## Untersuchen der Auswirkung von Parametern 

- viele komplexere Algorithmen haben **Parameter**, die ihr Verhalten beeinflussen, z.B.
   - Startort beim Nearest Neighbor
   - Anzahl an betrachteten Knoten beim Simplified Rollout
   - Anzahl der Lookahead-Steps beim Multistep Looahead
- oftmals ist es interessant, die Auswirkung der wesentlichen Parameter auf Ergebnis und Laufzeit zu untersuchen!



><div class="alert alert-block alert-info">
<b>Wie würden Funktionen aussehen, die die Auswirkung der Anzahl der betrachteten Nachbarn beim simplified Rollout für mehrere Instanzen untersuchen und die Resultate als Liste von `dicts` zurückgeben?</b></div>  

><div class="alert alert-block alert-info">
<b>Wie würden Funktionen aussehen, die die Auswirkung der Anzahl der betrachteten Nachbarn und der Lookahead-Steps beim simplified multistep lookahead für mehrere Instanzen untersuchen und die Resultate als Liste von `dicts` zurückgeben?</b></div>  

## Zusammenfassung

Heute haben wir einige Techniken zum Experimentieren kennengelernt:
- Aufrufen mehrerer Instanzen und Algorithmen
- Datenstrukturen für Ergebnisse
- Aufbereiten der Ergebnisse mit data frames
- Speichern von Ergebnissen in Dateien
- Auslesen von Ergebnissen aus Dateien


**Bitte beachten Sie:**
- Sie können natürlich auch anders arbeiten, z.B. direkt in Dateien schreiben
- Hier haben wir nur zwei Instanzen genutzt - es wäre schön, wenn Sie (deutlich) mehr Instanzen für Ihre Experimente nutzen würden!